In [17]:
import os
from getpass import getpass

import pandas as pd
from dotenv import load_dotenv
from IPython.display import Audio
from langchain_community.chat_models import ChatPerplexity
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

pplx_api_key = os.getenv("PPLX_API_KEY")
# model_name = "llama-3-8b-instruct"
model_name = "mixtral-8x7b-instruct"

In [18]:
chat = ChatPerplexity(temperature=0, model=model_name, api_key=pplx_api_key)
whisper_medium_results = pd.read_parquet("results/whisper_medium_results_df.parquet")
# whisper_medium_results = pd.read_parquet("results/wav2vec_results_df.parquet")

In [19]:
# system = """
# Summarize the audio transcript below. 
# If the transcript cannot be summarized using the information provided, respond with 'Could not summarize.'

# Context: You are a professional tasked with summarizing audio message transcripts. 
# You always aim to provide concise and accurate summaries of the transcript.
# But you should not add any information that is not present in the transcript.
# The summary should be in the form of a few sentences that capture the main points of the transcript.

# """

# human = """--- Please summarize this audio transcript:

# {text}
# ---

# Summarization: 

# """

In [20]:
system = """
Fasse das unten stehende Audiotranskript zusammen. 
Wenn das Transkript mit den angegebenen Informationen nicht zusammengefasst werden kann, antworte mit "Kann nicht zusammengefasst werden!".

Kontext: Sie sind ein Spezialist, der damit beauftragt ist, Transkripte von Audionachrichten zusammenzufassen. 
Dein Ziel ist es immer, präzise und genaue Zusammenfassungen des Transkripts zu erstellen.
Du solltest jedoch keine Informationen hinzufügen, die nicht in dem Transkript enthalten sind.
Die Zusammenfassung sollte aus wenigen Sätzen bestehen, die die wichtigsten Punkte der Audionachricht wiedergeben.
Ignoriere Rechtschreibfehler oder Grammatikfehler in der Audionachricht und verbessere diese in der Zusammenfassung.
Erwähne nicht, dass es sich um eine Zusammenfassung handelt oder dass du ein Transkript zusammenfasst.
Fasse es aus der Perspektive des Sprechers zusammen.

"""

human = """
--- Bitte fasse dieses Audiotranskript zusammen:

{text}
---

Zusammenfassung: 
"""

In [21]:
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
chain = prompt | chat

In [22]:
def get_summary(decoded_output):
    response = chain.invoke({"text": f"{decoded_output}"})
    return response.content

In [23]:
# get a random transcript
sample = whisper_medium_results.sample(1)
audio_path = sample["audio_path"].values[0]
decoded_output = sample["decoded_output"].values[0]

display(Audio(audio_path))
words_per_line = 10
splitted_transcript = decoded_output.split()
summary = get_summary(decoded_output)
splitted_summary = summary.split()

print("Transcript:")
for i in range(0, len(splitted_transcript), words_per_line):
    print(" ".join(splitted_transcript[i : i + words_per_line]))
    
print("\nSummary:")
for i in range(0, len(splitted_summary), words_per_line):
    print(" ".join(splitted_summary[i : i + words_per_line]))

Transcript:
Als wenn das noch nicht genug wäre, gibt es außerdem
erst mal nur eine provisorische Behausung. Das hatte ich mir
doch alles ganz anders vorgestellt, erinnert sich Ever. Einige Wochen
lang hätten sie so gelebt, meint die Rapperin, bis die
beiden schließlich in einem Frauenhaus unterkommen.

Summary:
Ever, die Rapperin, hatte sich das Leben zunächst anders vorgestellt.
Sie mussten vorübergehend in einer provisorischen Unterkunft leben, bevor sie
in ein Frauenhaus ziehen konnten, wo sie einige Wochen blieben.
